# 일등이조 (신경민)
# 북한기사 크롤링

In [3]:
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException, ElementNotInteractableException
from selenium.webdriver.common.action_chains import ActionChains
from bs4 import BeautifulSoup
import time

----

## 1. 북한 기사 동적 크롤링

- 사이트에서 날짜/검색어를 입력하여 url주소 가져오기
- ChromeDriver 실행 후 해당 URL 열기
- 팝업 발생 시 `ESC` 키 입력으로 닫기
- '더보기' 버튼이 보이지 않는 경우 대비해 첫 클릭 전 스크롤 조정
- `while` 반복문을 통해 '더보기' 버튼이 보일 때마다 클릭
- 버튼이 클릭 가능하도록 `scrollIntoView()` 후 살짝 스크롤 조정
- 버튼이 더 이상 없으면 `break`로 반복 종료
- 예외 발생 시 try-except로 처리하여 프로그램 중단 방지

In [2]:
service = Service(executable_path=ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)
url = 'https://kcnawatch.org/?s=&start=01-07-2011&end=30-09-2011'
driver.get(url)

actions = ActionChains(driver)
actions.send_keys(Keys.ESCAPE).perform()
time.sleep(1)

try:
    more_button = driver.find_element(By.CSS_SELECTOR, 'a.more-full-link')
    driver.execute_script("arguments[0].scrollIntoView();", more_button)
    time.sleep(2)
    current_scroll = driver.execute_script("return window.scrollY;")
    scroll_to = current_scroll - 300  
    driver.execute_script(f"window.scrollTo(0, {scroll_to});")
    time.sleep(2)
    more_button.click()
    time.sleep(2)
except Exception:
    print('첫 번째 클릭 실패 → 넘어감')

while True:
    try:
        more_button = driver.find_element(By.CSS_SELECTOR, 'a.more-full-link')
        
        if more_button.is_displayed():
            driver.execute_script("arguments[0].scrollIntoView(true);", more_button)
            time.sleep(2)
            current_scroll = driver.execute_script("return window.scrollY;")
            scroll_to = current_scroll - 300  
            driver.execute_script(f"window.scrollTo(0, {scroll_to});")
            time.sleep(2)
            driver.execute_script("arguments[0].click();", more_button)
            print(f'More Articles 클릭')
            time.sleep(2)
        else:
            print('버튼이 더 이상 보이지 않음 → 종료')
            break
    except (NoSuchElementException, ElementClickInterceptedException, ElementNotInteractableException):
        print('버튼 없음 또는 클릭 불가 → 종료')
        break

버튼이 더 이상 보이지 않음 → 종료


## 2. 기사 제목, 연도, URL 수집

- `html = driver.page_source`로 현재 페이지 소스 확보  
- BeautifulSoup으로 `search_wrapper` 영역의 기사 목록 파싱  
- `ESCAPE` 키를 눌러 팝업 대비  
- 각 기사에서 다음 정보 추출 후 리스트에 저장  
  - `name_list`: 기사 제목  
  - `year_list`: 기사 연도 (`<span>` 태그에서 추출)  
  - `url_list`: 기사 상세 주소 (`href` 중 `'newstream'` 포함된 것)  
- 추출된 데이터를 print()로 확인

In [11]:
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
actions.send_keys(Keys.ESCAPE).perform()
time.sleep(1)
articles = soup.find('div', {'id': 'search_wrapper'})
article_list = articles.find_all('div', {'class': 'article-desc'})

name_list = []
year_list = []
url_list = []

for article in article_list:
    span_tag = article.find('span')
    if span_tag:
        year_list.append(span_tag.text.strip())
for article in article_list:
    a_tag = article.find('a')
    if a_tag:
        name_list.append(a_tag.text.strip())
for article in article_list:
    a_tags = article.find_all('a')
    url_found = False
    for a_tag in a_tags:
        if a_tag.has_attr('href') and 'newstream' in a_tag.get('href'):
            url_list.append(a_tag.get('href'))
            url_found = True
            break
    if not url_found:
        url_list.append('')

for i in range(len(name_list)):
    print(f'제목: {name_list[i]}, 연도: {year_list[i]}, URL: {url_list[i]}')

제목: KCNA Delegation Comes back Home from US, 연도: July 01, 2011, URL: https://kcnawatch.xyz/newstream/1451890016-8404287/kcna-delegation-comes-back-home-from-us
제목: DPRK-made Clothes Gain in Popularity, 연도: July 01, 2011, URL: https://kcnawatch.xyz/newstream/1451890038-949460579/dprk-made-clothes-gain-in-popularity
제목: S. Korean Citizen Who Praised DPRK Sentenced to Prison Term, 연도: July 01, 2011, URL: https://kcnawatch.xyz/newstream/1451890053-974465869/s-korean-citizen-who-praised-dprk-sentenced-to-prison-term
제목: Heat Hits European Countries, 연도: July 01, 2011, URL: https://kcnawatch.xyz/newstream/1451890066-309943280/heat-hits-european-countries
제목: Belorussian President Blasts West's Sanctions, 연도: July 01, 2011, URL: https://kcnawatch.xyz/newstream/1451890079-864802237/belorussian-president-blasts-wests-sanctions
제목: Rodong Sinmun Calls for Throwing Group of Traitors Overboard, 연도: July 01, 2011, URL: https://kcnawatch.xyz/newstream/1451890090-617735247/rodong-sinmun-calls-for-thr

## 3. 기사 본문 가져오기 & 리스트 저장

- `url_list`에 담긴 각 기사 상세 URL을 순차적으로 열어 크롤링  
- `driver.page_source`로 현재 페이지의 HTML 소스를 가져옴  
- BeautifulSoup으로 `'article-content'` 클래스를 찾아 기사 본문 추출  
- 본문 텍스트는 줄바꿈(`\n`) 포함하여 정제 후 리스트에 추가  
- 본문이 없는 경우 `"본문 없음"`을 리스트에 추가하여 누락 방지

In [7]:
from selenium import webdriver
from bs4 import BeautifulSoup
con_list = []
driver = webdriver.Chrome()
for url in url_list:
    driver.get(url)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    contents = soup.find('div', {'class': 'article-content'})
    if contents:
        text = contents.get_text(separator='\n').strip()
        con_list.append(text)
    else:
        con_list.append('본문 없음')


## 4. DataFrame으로 만들고 csv저장

In [15]:
df = pd.DataFrame({
    'Date': year_list,
    'Title': name_list,
    'Content':con_list
})
df.to_csv('data/16_북한기사_전체_2.csv', index=False, encoding='utf-8-sig')

## 5. 크롤링 실패 & 리트라이 전략

- 크롤링 중 더보기 버튼 클릭 과정에서 페이지 로딩 지연이 자주 발생함  
- 단순히 `time.sleep()` 시간을 늘리는 방식은 페이지 속도에 따라 실패 가능성이 있음  
- 로딩이 일정 시간 내 완료되지 않으면 중단 후 재시도하는 방식이 더 신뢰성 있음  
- 따라서 `while` 루프를 통해 더보기 버튼이 나타날 때까지 반복 클릭 시도  
- 클릭이 안되거나 버튼이 더 이상 보이지 않으면 `break`로 안전하게 종료  
- `try-except` 블록으로 예외 처리하여 오류 발생 시에도 프로그램이 멈추지 않도록 설계

In [10]:
while True:
    try:
        more_button = driver.find_element(By.CSS_SELECTOR, 'a.more-full-link')
        
        if more_button.is_displayed():
            driver.execute_script("arguments[0].scrollIntoView(true);", more_button)
            time.sleep(2)
            current_scroll = driver.execute_script("return window.scrollY;")
            scroll_to = current_scroll - 300  
            driver.execute_script(f"window.scrollTo(0, {scroll_to});")
            time.sleep(2)
            driver.execute_script("arguments[0].click();", more_button)
            print(f'More Articles 클릭')
            time.sleep(2)
        else:
            print('버튼이 더 이상 보이지 않음 → 종료')
            break
    except (NoSuchElementException, ElementClickInterceptedException, ElementNotInteractableException):
        print('버튼 없음 또는 클릭 불가 → 종료')
        break

More Articles 클릭
More Articles 클릭
More Articles 클릭
More Articles 클릭
More Articles 클릭
More Articles 클릭
More Articles 클릭
More Articles 클릭
More Articles 클릭
More Articles 클릭
More Articles 클릭
More Articles 클릭
More Articles 클릭
More Articles 클릭
More Articles 클릭
More Articles 클릭
More Articles 클릭
More Articles 클릭
More Articles 클릭
More Articles 클릭
More Articles 클릭
More Articles 클릭
More Articles 클릭
More Articles 클릭
More Articles 클릭
More Articles 클릭
More Articles 클릭
More Articles 클릭
More Articles 클릭
More Articles 클릭
More Articles 클릭
More Articles 클릭
More Articles 클릭
More Articles 클릭
More Articles 클릭
More Articles 클릭
More Articles 클릭
More Articles 클릭
More Articles 클릭
More Articles 클릭
More Articles 클릭
More Articles 클릭
More Articles 클릭
More Articles 클릭
More Articles 클릭
More Articles 클릭
More Articles 클릭
More Articles 클릭
More Articles 클릭
More Articles 클릭
More Articles 클릭
More Articles 클릭
More Articles 클릭
More Articles 클릭
More Articles 클릭
More Articles 클릭
More Articles 클릭
More Articles 클릭
More Articles 

----